# Initialisation

In [240]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour Windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.optimize import curve_fit
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#import tifffile
#%matplotlib tk #pour Linux

In [241]:
import scipy.ndimage as ndimage
from sklearn.cluster import DBSCAN

# Importer fichier

In [242]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_10um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [243]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


62

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [244]:
frame_index = 0
first_frame = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    first_frame += 1
    print(frame_index)




# Traitement d'image

In [245]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)


# Sélection du point à tracker

In [246]:
def crop(img, x, y, crop_size_x, crop_size_y):  
    x_start = int(x - crop_size_x // 2)
    x_end = int(x + crop_size_x // 2)
    y_start = int(y - crop_size_y // 2)
    y_end = int(y + crop_size_y // 2)

    if x_start < 0:
        x_start = 0
    if x_end > 1440:
        x_end = 1440
    if y_start < 0:
        y_start = 0
    if y_end > 1080:
        y_end = 1080

    return img[y_start:y_end, x_start:x_end]

In [247]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, origin='lower', cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze_x = 250
crop_sze_y = 300

Please click on the point you want to select.
Selected point: (538.331168831169, 917.6168831168831)


Debogueur

In [248]:
def visionneur(frame, titre):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title(titre)
    plt.colorbar()  
    plt.show()

# Fit gaussien sur le point sélectionné

In [249]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [250]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [251]:
def localisateur_gaussien(intensity_grid, centre):
    x = np.arange(intensity_grid.shape[1])
    y = np.arange(intensity_grid.shape[0])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    print(max_idx)
    initial_x0 = x[max_idx[1]]
    initial_y0 = y[max_idx[0]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=5,
        sigma_y=5,
        offset=3
    )

    # Ajouter des bornes sur les paramètres x0 et y0
    params['x0'].min = 0    # Limite inférieure pour x0
    params['x0'].max = intensity_grid.shape[1] - 1  # Limite supérieure pour x0
    params['y0'].min = 0    # Limite inférieure pour y0
    params['y0'].max = intensity_grid.shape[0] - 1  # Limite supérieure pour y0

    # Ajouter des bornes sur les paramètres sigma
    params['sigma_x'].min = 1  # Limite inférieure de sigma_x
    params['sigma_x'].max = 40 # Limite supérieure de sigma_x
    params['sigma_y'].min = 1  # Limite inférieure de sigma_y
    params['sigma_y'].max = 40 # Limite supérieure de sigma_y

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))
    
    #print(f'position relative de gauss: {result.params['x0'].value}, {result.params['y0'].value}')
    
    x_position = result.params['x0'].value + centre[0] - crop_sze_x//4
    y_position = result.params['y0'].value + centre[1] - crop_sze_y//4

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [252]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Identifier la particule et donner sa nouvelle position

In [253]:
def is_within_bounds(position):
    """
    Vérifie si la position est à l'intérieur des limites du cadre (100 à 1300 en x et 100 à 980 en y).
    """
    x, y = position

    # Définir les bornes du cadre
    x_min, x_max = 100, 1300
    y_min, y_max = 100, 980

    # Vérifier si la position est dans les bornes du cadre
    if x_min <= x <= x_max and y_min <= y <= y_max:
        return True
    else:
        return False


In [254]:
def empreinte_digitale(image, threshold=50, eps=3, min_samples=1):
    # 1. Observer chaque particule approximativement
    #neighborhood_size = 3  
    #local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    visionneur(image, f'particule choisie')

    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)
    size=0

    # 2. Déterminer l'empreinte digitale de notre particule
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')

            if cluster_points.shape[0]>size:
                size = cluster_points.shape[0]
                luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])

    seuil=np.max(image)*0.75
    print(f'seuil:{seuil}, taille:{size} et luminosité: {luminosity}')

    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title(f"Empreinte digitale de la particule, lum={luminosity} et taille={size}")
    plt.legend()
    plt.colorbar()  
    plt.show()

    return size, luminosity, seuil

In [255]:
def detect_nbre_particles(image, threshold, eps=9, min_samples=1):
    # 1. Appliquer un filtre de voisinage pour détecter les maxima locaux
    neighborhood_size = 3  # Taille du voisinage pour détecter les maxima locaux
    local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    # 2. Comparer l'image originale et les maxima locaux pour identifier les vrais maxima
    maxima = (image == local_max) & (image > threshold)

    # 3. Extraire les coordonnées des maxima (particules)
    coordinates = np.column_stack(np.where(maxima))  # Extraire les indices des pixels maximaux
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        visionneur(image, f'détection du nombre de particules')

        return [], image

    # 4. Appliquer DBSCAN pour regrouper les particules proches (cluster les maxima détectés)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    #5. Déterminer le nombre de particules
    unique_labels = np.unique(labels)
    nb_particules = len(unique_labels)
        
    # # 6. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    # plt.imshow(image, origin='lower', cmap='gray')
    # plt.title("Détection des particules")
    
    positions=[]
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            # plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')
        positions.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))
        

    # plt.colorbar()  
    # plt.legend()
    # plt.show()

    # Retourner les coordonnées des particules filtrées
    return nb_particules, positions

In [256]:
def identificateur_particles(image, ref_size, ref_luminosity, threshold=50, eps=9, min_samples=1):
    # 1. Obtenir le plus d'info sur la particule
    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Mauvais re-crop empêche identification de la particule.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Parcourir chaque particule et identifier laquelle est celle qu'on suit
    distances = []
    position_particules = []

    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')

            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
            size_distance = abs(size - ref_size)  
            luminosity_distance = abs(luminosity - ref_luminosity) 
            
            #print(f'diff_taille: {size_distance} et diff_lum: {luminosity_distance}') 
            
            combined_distance = size_distance + luminosity_distance
            distances.append(combined_distance)
            position_particules.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))

    ecart = np.min(distances)
    position_best_particle = position_particules[np.argmin(distances)]  
    
    #print(f'ecrat: {ecart}')
    
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Identification des particules")
    plt.scatter(position_best_particle[0], position_best_particle[1], color='r', label=f'Particule sélectionnée')
    plt.legend()
    plt.colorbar()  
    plt.show()

    return position_best_particle, ecart

# Faire le crop et fit

In [257]:
def particle_tracker_simple(image, x, y):
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    #Gérer plus qu'une particule
    cropped_img = np.array(cropped_img)
    max_index = np.argmax(cropped_img)
    max_coords = np.unravel_index(max_index, cropped_img.shape)

    nouveau_x = x - crop_sze_x // 2 + max_coords[1]
    nouveau_y = y - crop_sze_y // 2 + max_coords[0]
    
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x, crop_sze_y)    # Re-crop autour d'une seule particule

    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

In [258]:
#Placés ici pour pouvoir y faire référence dans particule_tracker
position_list=[[x,y]]
sigma_list = []
crop_frames = []
big_frames = []

In [259]:
def particle_tracker(imge, x, y, taille_initiale, luminosité_initiale, seuil):
    # 1. Ce qu'on voit près de l'ancienne position
    image = denoise(imge)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    # 2. Déterminer combien il y a de particules et laquelle est la notre
    # Utiliser une reconnaissance de particules et si plus qu'une, alors clic pour choisir
    nb_part, coordonnées = detect_nbre_particles(cropped_img, seuil)

    if nb_part == 1:
        cropped_img = np.array(cropped_img)
        max_index = np.argmax(cropped_img)
        max_coords = np.unravel_index(max_index, cropped_img.shape)

        nouveau_x = x - crop_sze_x // 2 + max_coords[1] #Oui c'est inversé à cause du unravel juste avant
        nouveau_y = y - crop_sze_y // 2 + max_coords[0]
    elif  (nb_part == 0) or (nb_part == []):
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {len(position_list)-1}: Select a point")

        # Ajouter une croix rouge à la position donnée
        cx, cy = position_list[len(position_list)-1]
        ax.plot(cx, cy, 'rx', markersize=10) 
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({nouveau_x}, {nouveau_y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    else:
        imperfections, emplacements = [] , []
        for num_particule in range(nb_part):
            x_identifier=x - crop_sze_x // 2 + coordonnées[num_particule][0]
            y_identifier=y - crop_sze_y // 2 + coordonnées[num_particule][1]
            crop_pour_identifier = crop(image, x_identifier, y_identifier, crop_sze_x//2, crop_sze_x//2)
            emplacement, ecart = identificateur_particles(crop_pour_identifier, taille_initiale, luminosité_initiale)
            imperfections.append(ecart)
            emplacements.append(emplacement)
        qualité = np.min(imperfections)
        print(f'écart:{qualité}')
        max_coords = emplacements[np.argmin(imperfections)]  
        nouveau_x = x_identifier - crop_sze_x // 4 + max_coords[0]
        nouveau_y = y_identifier - crop_sze_y // 4 + max_coords[1]
    
    # Vérification s'il fout le camp 
    dis = np.sqrt((position_list[-1][0]-nouveau_x)**2+(position_list[-1][1]-nouveau_y)**2)
    if dis>100:
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {len(position_list)-1}: Problème d'identification, nouvelle pos à {dis}, select a point")

        print('problème d''identification')
        # Ajouter une croix rouge à la position donnée
        cx, cy = position_list[-1]
        ax.plot(cx, cy, 'rx', markersize=10) 
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({nouveau_x}, {nouveau_y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    
    # 3. Crop atour de notre particule et fit dessus
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x//2, crop_sze_y//2)    # Re-crop autour d'une seule particule


    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]
    #print(f'position estimée par le fit: {x_new}, {y_new}')

    #Affiche l'accord entre le fit gaussien et le blob (pour comparer)
    # x_range = np.arange(crop_sze_x) - crop_sze_x//2 
    # y_range = np.arange(crop_sze_y) - crop_sze_y//2
    # x_position = x_range + nouveau_x  
    # y_position = y_range + nouveau_y
    # plt.imshow(second_crop, origin='lower', cmap='gray', extent=[x_position.min(), x_position.max(), y_position.min(), y_position.max()])
    # plt.title("positionnement de la particule")
    # plt.scatter(x_new, y_new, color='r', label=f'Position estimée par fit gaussien {x_new}, {y_new}')
    # plt.scatter(nouveau_x, nouveau_y, color='r', marker='+', label=f'Position meilleure particule')
    # plt.legend()
    # plt.colorbar()  
    # plt.show()

    return [result_fit, second_crop, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [260]:
taille_initiale, luminosité_initiale, seuil = empreinte_digitale(crop(img,x,y,crop_sze_x*0.8,crop_sze_y*0.5))
print(f'position initiale: {position_list}')

for i in range(first_frame,actual_frames):
    tiff.seek(i)
    if np.sum(np.array(tiff))==0:
        position_list.append(np.array([np.nan,np.nan]))
        sigma_list.append(np.array([np.nan,np.nan]))
        print(f'frames vide:{i}')
    else:
        data = particle_tracker(img, x, y, taille_initiale, luminosité_initiale, seuil)
        position_list.append(data[2])
        x,y = position_list[i-first_frame+1]
        sigma_list.append(data[3])
        tiff.seek(i)
        img = np.array(tiff)
        big_frames.append(img)
        crop_frames.append(data[1])
    print(f'liste des positions frame {i}:{position_list}')
    if is_within_bounds(position_list[-1]):
        pass
    else:
        break



seuil:103.5, taille:2600 et luminosité: 84.13576923076923
position initiale: [[538.331168831169, 917.6168831168831]]
écart:487.56413269796565
(86, 62)
liste des positions frame 0:[[538.331168831169, 917.6168831168831], (536.8685106918658, 916.3618347567535)]
(75, 62)
liste des positions frame 1:[[538.331168831169, 917.6168831168831], (536.8685106918658, 916.3618347567535), (537.965826125619, 914.3660634800347)]
(75, 62)
liste des positions frame 2:[[538.331168831169, 917.6168831168831], (536.8685106918658, 916.3618347567535), (537.965826125619, 914.3660634800347), (541.5874907487992, 894.5607595895148)]
(75, 62)
liste des positions frame 3:[[538.331168831169, 917.6168831168831], (536.8685106918658, 916.3618347567535), (537.965826125619, 914.3660634800347), (541.5874907487992, 894.5607595895148), (546.1341759264443, 866.1904183037063)]
(75, 62)
liste des positions frame 4:[[538.331168831169, 917.6168831168831], (536.8685106918658, 916.3618347567535), (537.965826125619, 914.3660634800347

In [261]:
# posit=[[538.331168831169, 917.6168831168831], (536.8685106918658, 916.3618347567535), (537.965826125619, 914.3660634800347), (541.5874907487992, 894.5607595895148), (546.1341759264443, 866.1904183037063), (548.1077517019759, 846.9710937445128), (546.8232392054424, 825.9390319040917), (546.8958574112244, 796.6092383595795), (555.8906872742591, 774.8889278483235), (562.9205798692265, 753.5192475725455), (566.0351617511109, 723.6954011054476), (568.9812587099784, 701.9135200722601), (570.7904800338555, 682.5705801545878), (572.0253073151746, 655.3347893201487), (571.5486097631067, 636.4814203155963), (570.7693680423006, 612.1606317384776), (572.635611207188, 590.2414713373705), (574.1656525169153, 569.8341360146392), (576.1667801953998, 542.3614237152334), (578.1282804848723, 523.9289474385789), (580.6410750301233, 501.02392804871215), np.array([np.nan, np.nan]), np.array([np.nan, np.nan]), (582.1997420974418, 475.1007414476379), (588.3930746876018, 403.8952843814146), (593.1500882830323, 384.6824832070706), np.array([np.nan, np.nan]), (596.4757194490121, 359.30340110825495), np.array([np.nan, np.nan]), np.array([np.nan, np.nan]), (609.7556374695245, 314.51007158859056), (612.9522348007297, 244.81113887227002), np.array([np.nan, np.nan]), (614.4768211936405, 219.95444189187896), np.array([np.nan, np.nan]), (616.7586989569447, 172.99719628876824), (617.8017705405762, 128.25464830896183), (616.7644015875331, 106.6328481429959), (614.7673256245066, 31.14671524096383), (613.021732845815, 14.813363748488584)]
# # Créer une image vide de taille appropriée (par exemple, 400x400 pixels)
# image = np.zeros((1080, 1440))

# # Placer les points du vecteur 'posit' sur l'image en utilisant la valeur maximale de 1
# for (x, y) in posit:
#     # On arrondit les coordonnées pour qu'elles correspondent à des indices valides dans l'image
#     image[int(round(y)), int(round(x))] = 1

# # Afficher l'image avec imshow
# plt.imshow(image, cmap='gray', origin='lower')
# plt.title('Points sur l\'image')

# # Afficher les points avec un scatter (en superposition sur l'image)
# x_coords, y_coords = zip(*posit)
# plt.scatter(x_coords, y_coords, color='red', label='Points', marker='x')

# # Ajouter une légende
# plt.legend()

# # Afficher l'image
# plt.show()

In [262]:
pixel_size = 3.45 # Taille absolue
f2 = 150  # Focale de L2
M_theo = 20  # Magnification of the objective
pxl = pixel_size / (f2 * M_theo / 160)  # Pixel size in um dans le plan de la particule

sigma_arr=np.array(sigma_list)* pxl * 10**(-6)
position_arr=np.array(position_list)* pxl * 10**(-6)
print(position_arr)

[[9.90529351e-05 1.68841506e-04]
 [9.87838060e-05 1.68610578e-04]
 [9.89857120e-05 1.68243356e-04]
 [9.96520983e-05 1.64599180e-04]
 [1.00488688e-04 1.59379037e-04]
 [1.00851826e-04 1.55842681e-04]
 [1.00615476e-04 1.51972782e-04]
 [1.00628838e-04 1.46576100e-04]
 [1.02283886e-04 1.42579563e-04]
 [1.03577387e-04 1.38647542e-04]
 [1.04150470e-04 1.33159954e-04]
 [1.04692552e-04 1.29152088e-04]
 [1.05025448e-04 1.25592987e-04]
 [1.05252657e-04 1.20581601e-04]
 [1.05164944e-04 1.17112581e-04]
 [1.05021564e-04 1.12637556e-04]
 [1.05364952e-04 1.08604431e-04]
 [1.05646480e-04 1.04849481e-04]
 [1.06014688e-04 9.97945020e-05]
 [1.06375604e-04 9.64029263e-05]
 [1.06837958e-04 9.21884028e-05]
 [           nan            nan]]


In [275]:
def calculate_msd_with_uncertainty(position_arr, delta_x, delta_y):
    """
    Calcule les MSD avec propagation des incertitudes analytiques.
    """
    if position_arr[0:].shape[0] != delta_x[0:].shape :
        position_arr=position_arr[:len(position_arr)-1]

    # Traitement Lucien
    diff_pairs = np.zeros_like(position_arr[1:])
    #diff_pairs = np.zeros(len(position_arr[1:]))  # Un tableau 1D pour les différences
    dx = np.zeros_like(delta_x[1:])
    dy = np.zeros_like(delta_y[1:])

    # Masque pour détecter les NaN
    nan_mask_pos = np.isnan(position_arr[1:]) | np.isnan(position_arr[:-1])
    nan_mask_delta = np.isnan(delta_x[1:]) | np.isnan(delta_x[:-1])
    
    # Calculer les différences uniquement où nan_mask est False
    valid_mask_pos = ~nan_mask_pos.any(axis=1)  # Inverser le masque pour obtenir les position_arr valides
    valid_mask_delta = ~nan_mask_delta
    
    diff_pairs[valid_mask_pos] = position_arr[1:][valid_mask_pos, :] - position_arr[:-1][valid_mask_pos, :]
    # Calculer les différences en x uniquement (prendre la première colonne des positions)
    #diff_pairs[valid_mask_pos] = position_arr[1:][valid_mask_pos, 0] - position_arr[:-1][valid_mask_pos, 0]

    vitesse_moyenne = np.mean(diff_pairs,axis=0)
    
    temps = np.arange(0,len(position_arr))  # vecteur de temps

    # Appliquer la soustraction terme à terme
    nouvelles_données = position_arr - vitesse_moyenne * temps[:, np.newaxis]    

    msd = []
    uncertainties = []
    for d in range(1, len(nouvelles_données)):
        diff_pairs = np.zeros_like(nouvelles_données[d:])
        #diff_pairs = np.zeros(len(position_arr[d:]))  # Un tableau 1D pour les différences
        dx = np.zeros_like(delta_x[d:])
        dy = np.zeros_like(delta_y[d:])

        # Masque pour détecter les NaN
        nan_mask_pos = np.isnan(nouvelles_données[d:]) | np.isnan(nouvelles_données[:-d])
        nan_mask_delta = np.isnan(delta_x[d:]) | np.isnan(delta_x[:-d])
        
        # Calculer les différences uniquement où nan_mask est False
        valid_mask_pos = ~nan_mask_pos.any(axis=1)  # Inverser le masque pour obtenir les position_arr valides
        valid_mask_delta = ~nan_mask_delta
        
        diff_pairs[valid_mask_pos] = nouvelles_données[d:][valid_mask_pos, :] - nouvelles_données[:-d][valid_mask_pos, :]
        # Calculer les différences en x uniquement (prendre la première colonne des positions)
        #diff_pairs[valid_mask_pos] = position_arr[d:][valid_mask_pos, 0] - position_arr[:-d][valid_mask_pos, 0]

        distances_squared = np.sum(diff_pairs**2)#, axis=1)
        msd.append(np.mean(distances_squared))  
        
        dx[valid_mask_delta] = delta_x[d:][valid_mask_delta] + delta_x[:-d][valid_mask_delta]
        dy[valid_mask_delta] = delta_y[d:][valid_mask_delta] + delta_y[:-d][valid_mask_delta]
        term_x = 2 * (diff_pairs[:, 0]**2) * (dx**2)
        term_y = 2 * (diff_pairs[:, 1]**2) * (dy**2)

        #term_x = 2 * (diff_pairs**2) * (dx**2)  # Utilisation uniquement de diff_pairs pour x
        #term_y = np.zeros_like(term_x)  # Ne pas utiliser dy car on ne calcule pas la distance en y
        
        # Propagation des incertitudes
        total_uncertainty = np.mean(term_x + term_y)
        uncertainties.append(np.sqrt(total_uncertainty))

    return np.array(msd), np.array(uncertainties)

calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

(array([4.04904956e-11, 9.37733161e-11, 1.02055572e-10, 1.11111809e-10,
        1.38889118e-10, 1.56939419e-10, 1.65351152e-10, 1.86139319e-10,
        1.95679065e-10, 1.96301950e-10, 2.07044912e-10, 2.06166481e-10,
        1.97696821e-10, 1.90926401e-10, 1.65991995e-10, 1.40905610e-10,
        1.13390030e-10, 6.13755400e-11, 1.35570643e-11, 0.00000000e+00]),
 array([1.57654267e-11, 2.33973652e-11, 2.41103388e-11, 2.71709800e-11,
        2.96145890e-11, 3.29703950e-11, 3.55713341e-11, 3.82167967e-11,
        4.16063066e-11, 4.32739118e-11, 4.54784977e-11, 4.81438752e-11,
        4.90613108e-11, 4.96309766e-11, 4.93048651e-11, 4.86559705e-11,
        4.24728125e-11, 2.17849915e-11, 3.36220042e-12, 0.00000000e+00]))

# Résultats

In [276]:
# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])
print('coucou')
# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(msd_values) + 1) * 0.5

# Plotting the MSD
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, msd_values, yerr=uncertainties, fmt='o', label='MSD', color='blue')

# Customize the plot
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

coucou


In [277]:
# Fonction quadratique pour l'ajustement
def quadratic(x, a, b, c):
    return a * x**2 + b * x + c
def line(x, b,c):
    return b*x+c

# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

cropped_msd = msd_values[:5]
cropped_inc = uncertainties[:5]

# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(cropped_msd) + 1)

# Perform a quadratic fit using curve_fit, which gives both coefficients and covariance matrix
bounds = ([-np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf])
popt, pcov = curve_fit(line, time_intervals, cropped_msd, sigma=cropped_inc)

# popt contains the optimal coefficients
coeffs = popt

# pcov is the covariance matrix, which we can use to compute uncertainties
coeff_uncertainties = np.sqrt(np.diag(pcov))  # Incertitudes sur les coefficients sont les éléments diagonaux

# Affichage des coefficients séparément
print("Coefficients du polynôme ajusté :")
print(coeffs)

# Affichage de la matrice de covariance séparément
print("Matrice de covariance des coefficients :")
print(pcov)

# Création de la fonction quadratique de l'ajustement
quadratic_fit = np.poly1d(coeffs)

# Génération des valeurs ajustées pour afficher la courbe
fitted_msd = quadratic_fit(time_intervals)

# Affichage du MSD avec les barres d'erreur
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, cropped_msd, yerr=cropped_inc, fmt='o', label='MSD', color='blue')
plt.plot(time_intervals, fitted_msd, label='Quadratic Fit', color='red', linestyle='--')

# Personnalisation du graphique
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Afficher le graphique
plt.show()

# Affichage des incertitudes sur les coefficients
print(f"Incertitudes sur les coefficients : {coeff_uncertainties}")

# Calcul du coefficient de diffusion et de la taille de la particule
r = (4 * 1.38 * 10**-23 * 300 / (6 * np.pi * 10**(-3) * coeffs[0]))  # Coefficient de diffusion
r_uncertainty = (4 * 1.38 * 10**(-23) * 300 * coeff_uncertainties[0] / (6 * np.pi * 10**(-3) * (coeffs[0])**2))  # Coefficient de diffusion

# Affichage de la taille de la particule avec incertitude
print(f"Taille de la particule (m): {r:.2e} m, avec une incertitude : {r_uncertainty:.2e}")


Coefficients du polynôme ajusté :
[2.39132179e-11 2.44162004e-11]
Matrice de covariance des coefficients :
[[ 2.15696154e-23 -5.13397595e-23]
 [-5.13397595e-23  1.64992016e-22]]
Incertitudes sur les coefficients : [4.64431000e-12 1.28449218e-11]
Taille de la particule (m): 3.67e-08 m, avec une incertitude : 7.14e-09


In [278]:
#3.48315140e-12 -2.34531178e-13  2.27994622e-12
# Perform a quadratic fit using curve_fit, which gives both coefficients and covariance matrix
# bounds = ([-np.inf, 0, -np.inf], [np.inf, 1, np.inf])
popt, pcov = curve_fit(line, time_intervals, cropped_msd, sigma=cropped_inc)

# popt contains the optimal coefficients
coeffs = popt

# pcov is the covariance matrix, which we can use to compute uncertainties
coeff_uncertainties = np.sqrt(np.diag(pcov))  # Incertitudes sur les coefficients sont les éléments diagonaux

# Affichage des coefficients séparément
print("Coefficients du polynôme ajusté :")
print(coeffs)

# Affichage de la matrice de covariance séparément
print("Matrice de covariance des coefficients :")
print(pcov)

# Affichage du MSD avec les barres d'erreur
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, cropped_msd, yerr=cropped_inc, fmt='o', label='MSD', color='blue')

# Personnalisation du graphique
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Afficher le graphique
plt.show()

# Affichage des incertitudes sur les coefficients
print(f"Incertitudes sur les coefficients : {coeff_uncertainties}")

# Calcul du coefficient de diffusion et de la taille de la particule
r = (4 * 1.38 * 10**-23 * 300 / (6 * np.pi * 10**(-3) * coeffs[1]))  # Coefficient de diffusion
r_uncertainty = (4 * 1.38 * 10**(-23) * 300 * coeff_uncertainties[1] / (6 * np.pi * 10**(-3) * (coeffs[1])**2))  # Coefficient de diffusion

# Affichage de la taille de la particule avec incertitude
print(f"Taille de la particule (m): {r:.2e} m, avec une incertitude : {r_uncertainty:.2e}")


Coefficients du polynôme ajusté :
[2.39132179e-11 2.44162004e-11]
Matrice de covariance des coefficients :
[[ 2.15696154e-23 -5.13397595e-23]
 [-5.13397595e-23  1.64992016e-22]]
Incertitudes sur les coefficients : [4.64431000e-12 1.28449218e-11]
Taille de la particule (m): 3.60e-08 m, avec une incertitude : 1.89e-08


In [279]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Inverser l'axe Y
plt.gca().invert_yaxis()

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

# Afficher le graphique
plt.show()

## anim toutes les particules

In [280]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], origin='lower', cmap='gray', animated=True)


# Update function
def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
plt.show()

## anim crop

In [281]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], origin='lower', cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()